# AWSClient

In [ ]:
from collections import defaultdict

import pandas

from   datalabs.access.aws import AWSClient

## /snomed/maps/cpt

In [ ]:
def get_mapping_references_for_keyword(keyword, db):
    maps = None

    search_results = db.execute_statement(
        Statement=f"SELECT * FROM \"CPT-API-snomed-sbx\".\"SearchIndex\" WHERE sk = 'KEYWORD:{keyword}'",
        Limit=1000000
    )

    return search_results["Items"]

In [ ]:
def get_mappings_from_references(keyword_items, db):
    mappings = defaultdict(list)

    for search_item in keyword_items:
        mapping = get_mapping_from_reference(search_item['pk']['S'], db)

        mappings[mapping["pk"]["S"]].append(mapping)

    return mappings

In [ ]:
# TODO: handle DynamoDB pagination

def get_all_mappings(db):
    mappings = defaultdict(list)

    results = db.execute_statement(
        Statement=f"SELECT * FROM \"CPT-API-snomed-sbx\" WHERE begins_with(\"sk\", 'CPT:')"
    )
    
    for item in results["Items"]:
        mappings[item["pk"]["S"]].append(item)

    return mappings

In [ ]:
def generate_map(mapping_items):
    mappings = []
    concept = None

    for item in mapping_items:
        mapping = {key:value['S'] for key, value in item.items()}

        if not concept:
            concept = mapping["pk"]
            snomed_descriptor = mapping["snomed_descriptor"]

        mappings.append(
            dict(
                code=mapping["sk"].replace("CPT:", ""),
                descriptor=mapping["cpt_descriptor"],
                category=mapping["map_category"]
            )
        )

    return dict(
        concept=concept,
        descriptor=snomed_descriptor,
        mappings=mappings
    )

In [ ]:
def get_mapping_from_reference(pksk, db):
    pk = pksk.rsplit(':', 2)[0]   # CONCEPT:...
    sk = pksk.split(':', 2)[2]    # CPT:...

    concept = pk.replace("CONCEPT:", "")

    results = db.execute_statement(
        Statement=f"SELECT * FROM \"CPT-API-snomed-sbx\" WHERE pk = '{pk}' AND sk = '{sk}'"
    )

    return results["Items"][0]

## Use Case 1: Get Mappings by Keyword

In [ ]:
# TODO: handle response pagination
mapping_sets = {}

keyword = "discharge"
for keyword in keywords:
    with AWSClient("dynamodb") as db:
        mapping_references = get_mapping_references_for_keyword(keyword, db)

        mapping_set = get_mappings_from_references(mapping_references, db)
    
    mapping_sets.update(mapping_set)

maps = [generate_map(mappings) for mappings in mapping_set.values()]
maps

## Use Case 2: Get All Mappings (Default)

In [ ]:
# TODO: handle response pagination

with AWSClient("dynamodb") as db:
    mapping_set = get_all_mappings(db)

maps = [generate_map(mappings) for mappings in mapping_set.values()]
maps